Для работы скрипта необходимо скачать файл CATBOOST.z, приложенный в репозитории.

In [ ]:
import pandas as pd
import joblib
from sklearn.model_selection import RandomizedSearchCV
from statistics import mean
from catboost import CatBoostClassifier

In [ ]:
def submit():
    test = pd.read_parquet("https://www.dropbox.com/scl/fi/iy4il4b2248czc3bmd4kf/test.parquet?rlkey=d7z4q0wwelj0f7qpb1lj485pe&st=crox5j23&dl=1")
    test['dates'] = test['dates'].apply(lambda x: [pd.to_datetime(d) for d in x])
    new_test = pd.DataFrame({
        'id': test['id'].repeat(test['dates'].str.len()),
        'date': [date for dates in test['dates'] for date in dates],
        'value': [value for values in test['values'] for value in values]
    })
    
    def additional_features(df):
        df = df.set_index(['id', 'date']).unstack(fill_value=0).stack().reset_index()
        df['rolling_mean'] = df.groupby('id')['value'].transform(lambda x: x.rolling(window=3).mean())
        df['year'] = df['date'].dt.year 
        df['month'] = df['date'].dt.month 
        df['day_of_week'] = df['date'].dt.dayofweek
        df['lag_value_1'] = df['value'].shift(1)
        df['lag_value_2'] = df['value'].shift(2)
        df['lag_value_3'] = df['value'].shift(3)
        df.dropna(inplace=True)
        return df
        
    new_test = additional_features(new_test)
    new_test.reset_index(drop=True, inplace=True)
    RSCV = joblib.load('CATBOOST.z')
    test_predictions = RSCV.predict_proba(new_test)[:, 1]
    predictions = pd.DataFrame(data=test_predictions, columns=['score'])
    predictions['id'] = new_test['id']
    predictions = predictions.reindex(columns=['id', 'score'])
    submissions = predictions.groupby('id').mean().reset_index()
    submissions.to_csv("subtest.csv")

In [ ]:
submit()